In [59]:
#import libraries
import pandas as pd

# Reorganize world cup data

In [60]:
#import csv
worldBank = pd.read_csv('worldBank.csv')

#drop redundant columns
worldBank = worldBank.drop(columns='Indicator Code')
worldBank = worldBank.drop(columns='Country Code')

worldBank.head()

,Country Name,Indicator Name,1960,1961,1962,1963,1964,1965,1966,1967,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,Unnamed: 60
0,Arab World,% of females ages 15-49 having comprehensive c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,% of males ages 15-49 having comprehensive cor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,"Adolescent fertility rate (births per 1,000 wo...",133.555013,134.159119,134.857912,134.504576,134.105211,133.569626,132.675635,131.665502,...,49.999851,49.887046,49.781207,49.672975,49.536047,49.383745,48.796558,48.196418,NaN,NaN
3,Arab World,Adults (ages 15+) and children (0-14 years) li...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arab World,Adults (ages 15+) and children (ages 0-14) new...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
#wide to long
worldBankMelt = worldBank.melt(id_vars=['Country Name', 'Indicator Name'], var_name='Year', value_name='Value')
worldBankMelt.head()

,Country Name,Indicator Name,Year,Value
0,Arab World,% of females ages 15-49 having comprehensive c...,1960,NaN
1,Arab World,% of males ages 15-49 having comprehensive cor...,1960,NaN
2,Arab World,"Adolescent fertility rate (births per 1,000 wo...",1960,133.555013
3,Arab World,Adults (ages 15+) and children (0-14 years) li...,1960,NaN
4,Arab World,Adults (ages 15+) and children (ages 0-14) new...,1960,NaN


In [62]:
#multiindex pivot function
def multiindex_pivot(df, columns=None, values=None):
    #https://github.com/pandas-dev/pandas/issues/23955
    names = list(df.index.names)
    df = df.reset_index()
    list_index = df[names].values
    tuples_index = [tuple(i) for i in list_index] # hashable
    df = df.assign(tuples_index=tuples_index)
    df = df.pivot(index="tuples_index", columns=columns, values=values)
    tuples_index = df.index  # reduced
    index = pd.MultiIndex.from_tuples(tuples_index, names=names)
    df.index = index
    return df

#set index
worldBankIndex = worldBankMelt.set_index(['Country Name', 'Year'])

#multiindex pivot
worldBankIndex = multiindex_pivot(worldBankIndex, 'Indicator Name', 'Value')
worldBankIndex.head()

Indicator Name     % of females ages 15-49 having comprehensive correct knowledge about HIV (2 prevent ways and reject 3 misconceptions)  \
Country Name Year                                                                                                                          
Afghanistan  1960                                                NaN                                                                       
             1961                                                NaN                                                                       
             1962                                                NaN                                                                       
             1963                                                NaN                                                                       
             1964                                                NaN                                                                       

Indicator Name     % of males ages 15-49 having comprehensive correct knowledge about HIV (2 prevent ways and reject 3 misconceptions)  \
Country Name Year                                                                                                                        
Afghanistan  1960                                                NaN                                                                     
             1961                                                NaN                                                                     
             1962                                                NaN                                                                     
             1963                                                NaN                                                                     
             1964                                                NaN                                                                     

Indicator Name     AIDS estimated deaths (UNAIDS estimates)  \
Country Name Year                                             
Afghanistan  1960                                       NaN   
             1961                                       NaN   
             1962                                       NaN   
             1963                                       NaN   
             1964                                       NaN   

Indicator Name     ARI treatment (% of children under 5 taken to a health provider)  \
Country Name Year                                                                     
Afghanistan  1960                                                NaN                  
             1961                                                NaN                  
             1962                                                NaN                  
             1963                                                NaN                  
             1964                                                NaN                  

Indicator Name     Adolescent fertility rate (births per 1,000 women ages 15-19)  \
Country Name Year                                                                  
Afghanistan  1960                                            145.321               
             1961                                            145.321               
             1962                                            145.321               
             1963                                            145.321               
             1964                                            145.321               

Indicator Name     Adults (ages 15+) and children (0-14 years) living with HIV  \
Country Name Year                                                                
Afghanistan  1960                                                NaN             
             1961                                                NaN             
             1962                                                NaN             
 

# Build qualifier dictionary

In [88]:
footballResults = pd.read_csv('all_int_football_results.csv')

worldCup = footballResults[footballResults['tournament'] == 'FIFA World Cup']

worldCup['Year'] = worldCup['date'].str.split('-').str[0]

worldCup.head()

/Users/greggior/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,Year
1300,1930-07-13,Belgium,United States,0,3,FIFA World Cup,Montevideo,Uruguay,True,1930
1301,1930-07-13,France,Mexico,4,1,FIFA World Cup,Montevideo,Uruguay,True,1930
1302,1930-07-14,Brazil,Yugoslavia,1,2,FIFA World Cup,Montevideo,Uruguay,True,1930
1303,1930-07-14,Peru,Romania,1,3,FIFA World Cup,Montevideo,Uruguay,True,1930
1304,1930-07-15,Argentina,France,1,0,FIFA World Cup,Montevideo,Uruguay,True,1930


In [81]:
worldCupYears = worldCup['Year'].unique()
worldCupYears

array(['1930', '1934', '1938', '1950', '1954', '1958', '1962', '1966',
       '1970', '1974', '1978', '1982', '1986', '1990', '1994', '1998',
       '2002', '2006', '2010', '2014', '2018'], dtype=object)

In [82]:
allHome = list(worldCup['home_team'].unique())
allAway = list(worldCup['away_team'].unique())
allTeams = allHome + allAway
allQualTeams = set(allTeams)

print(len(allQualTeams))

In [85]:
qualDict = {}

for year in worldCupYears:
    home = list(worldCup[worldCup['Year']==year]['home_team'].unique())
    away = list(worldCup[worldCup['Year']==year]['away_team'].unique())
    teams = set(home + away)
    qualDict[year] = teams

# Differences in country names between datasets

In [86]:
worldBankCountries = list(worldBank['Country Name'].unique())
len(worldBankCountries)

258

In [87]:
for country in allQualTeams:
    if country not in worldBankCountries:
        print(country)

Yugoslavia
North Korea
Ivory Coast
Republic of Ireland
Egypt
Wales
Czechoslovakia
German DR
Slovakia
Scotland
DR Congo
Northern Ireland
South Korea
Russia
China PR
England
Iran


In [89]:
#function to rename countries in world cup dataset
def changeCountryName(country):
    if country == 'Yugoslavia':
        return
    if country == 'North Korea':
        return 
    if country == 'Ivory Coast':
        return 
    if country == 'Republic of Ireland':
        return
    if country == 'Egypt':
        return
    if country == 'Wales':
        return
    if country == 'Czechoslovakia':
        return
    if country == 'German DR':
        return 'Germany'
    if country == 'Slovakia':
        return
    if country == 'Scotland':
        return
    if country == 'DR Congo':
        return
    if country == 'Northern Ireland':
        return
    if country == 'South Korea':
        return
    if country == 'Russia':
        return
    if country == 'China PR':
        return 'China'
    if country == 'England':
        return
    if country == 'Iran':
        return